# Bài tập Thực hành Buổi 01

**Họ và tên:** Trương Hoàng Thông  
**MSSV:** 23210287

---
**Môn học:** Công Nghệ Phân Tích Dữ Liệu Lớn
**Giảng viên:** ThS. Nguyễn Hồ Duy Trí

---

## 1. Import thư viện

In [1]:
from pyspark.sql import SparkSession, functions as F
import math
import random

## 2. Tạo dãy số thực ngẫu nhiên và DataFrame

In [2]:
spark = SparkSession.builder.appName("Lab01_PySpark").getOrCreate()

numbers = [random.uniform(0, 100) for _ in range(10)]
print(f"Day so thuc: {numbers}")
df = spark.createDataFrame([(float(x),) for x in numbers], ["value"])


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/01 02:01:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Day so thuc: [22.648612431181046, 82.2494911779238, 78.05666092835338, 86.21396904999486, 98.75418444171784, 25.202660219413055, 47.13493768382394, 13.688899166855073, 97.18933163152694, 18.280502495569284]


## 3. Tính trung bình cộng và trung bình nhân

In [3]:
# Tinh trung binh cong
trung_binh_cong = df.agg(F.avg("value")).collect()[0][0]

# Tinh trung binh nhan, chi tinh neu tat ca cac so deu duong
if all(x > 0 for x in numbers):
    log_mean = df.agg(F.avg(F.log("value"))).collect()[0][0]
    trung_binh_nhan = math.exp(log_mean)
else:
    trung_binh_nhan = None

print(f"Trung binh cong: {trung_binh_cong}")
print(f"Trung binh nhan: {trung_binh_nhan}")

Trung binh cong: 56.94192492263593
Trung binh nhan: 45.221842245000644


## 4. Tính phương sai và độ lệch chuẩn

In [4]:
# Tinh phuong sai
phuong_sai = df.agg(F.variance("value")).collect()[0][0]
print(f"Phuong sai: {phuong_sai}")

# Tinh do lech chuan
do_lech_chuan = df.agg(F.stddev("value")).collect()[0][0]
print(f"Do lech chuan: {do_lech_chuan}")

Phuong sai: 1217.3772635541438
Do lech chuan: 34.89093383035404


## 5. Đọc dữ liệu phân số từ file và tính tổng

In [5]:
# Euclid, tim uoc so chung lon nhat
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

rdd = spark.sparkContext.textFile("data_phanso.txt")
day_phan_so = rdd.map(lambda line: tuple(map(int, line.strip().split("/"))))
tu_mau = day_phan_so.collect()

mau_so_chung = 1
for _, mau in tu_mau:
    mau_so_chung *= mau

tong_tu_so = 0
for tu, mau in tu_mau:
    tong_tu_so += tu * (mau_so_chung // mau)

ucln = gcd(tong_tu_so, mau_so_chung)
tong_tu_so //= ucln
mau_so_chung //= ucln

print(f"Tong day phan so: {tong_tu_so}/{mau_so_chung}")

Tong day phan so: 151/40
